In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# run environment

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.4)
sess_config = tf.ConfigProto(gpu_options=gpu_options)

In [3]:
def segment_(data,window_size = 40):
    segments = np.empty((0,1,window_size,3))
    labels = np.empty((0))
    for line in data:
        line = np.array(line).reshape(1,1,window_size,3)
        segments = np.vstack([segments,line])
    return segments

# init nn parameters
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

# conv
def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x,kernel_size,num_channels,depth):
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(depthwise_conv2d(x, weights),biases))
    
def apply_max_pool(x,kernel_size,stride_size):
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')

# load data

In [4]:
fault_label = {'1': 'info',
               '2': 'Critical',
               '3': 'error',
               '4': 'notice',
               '5': 'warning',
               '6': 'alert',
               '7': 'emergency'}

X, X_val, labels, labels_val = [], [], [], [] #validation set features

with open("data/msg_token_train.txt", 'rU') as f:
        res = list(f)
        for line in res:
            line.strip("\n")
            features = line.split("\t")
            while features.__contains__(""):
                features.remove("")
            for i in range(len(features)):
                features[i] = float(features[i])
            X.append(features)
            
with open("data/msg_label_train.txt", 'rU') as f:
        res = list(f)
        for line in res:
            labels.append(int(line.strip("\n")[0]))
            
with open("data/msg_token_test.txt", 'rU') as f:
        res = list(f)
        for line in res:
            line.strip("\n")
            features = line.split("\t")
            while features.__contains__(""):
                features.remove("")
            for i in range(len(features)):
                features[i] = float(features[i])
            X_val.append(features)
            
with open("data/msg_label_test.txt", 'rU') as f:
        res = list(f)
        for line in res:
            labels_val.append(int(line.strip("\n")[0]))

segments = segment_(X)
segments_val = segment_(X_val)
print(segments.shape)

labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)
labels_val = np.asarray(pd.get_dummies(labels_val), dtype = np.int8)
print(labels.shape)

train_x = segments.reshape(len(segments), 1, 40, 3)
train_y = labels
test_x = segments_val.reshape(len(segments_val), 1, 40, 3)
test_y = labels_val

# Separate our training data into test and training.
print("Separating data into 67% training set & 33% test set...")
train_x, test_x, train_y, test_y = cross_validation.train_test_split(
    train_x, train_y, test_size=0.33, random_state=33)#add random state here...
print("Dataset separated.\n")

(8360, 1, 40, 3)
(8360, 7)
Separating data into 67% training set & 33% test set...
Dataset separated.



# define parameters

In [5]:
input_height = 1
input_width = 40
num_channels = 3
num_labels = 7

batch_size = 128
kernel_size = 5
depth = 20
num_hidden = 1000

learning_rate = 0.0001
# iteration times
training_epochs = 100
lstm_size = 128

total_batches = train_x.shape[0] // batch_size

n_classes = 7
n_hidden = 128
n_inputs = 120

rnnW = {
    'hidden': tf.Variable(tf.random_normal([n_inputs, n_hidden])),
    'output': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
rnnBiases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden], mean=1.0)),
    'output': tf.Variable(tf.random_normal([n_classes]))
}

# build network

In [6]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

c = apply_depthwise_conv(X,kernel_size,num_channels,depth)
p = apply_max_pool(c,5,2)
c2 = apply_depthwise_conv(p,3,depth*num_channels,depth//10)
c2Reshape = tf.reshape(c2, [-1, 7, n_inputs])
shuff = tf.transpose(c2Reshape, [1, 0, 2])
shuff = tf.reshape(shuff, [-1, n_inputs])

# Linear activation, reshaping inputs to the LSTM's number of hidden:
hidden = tf.nn.relu(tf.matmul(shuff, rnnW['hidden']) + rnnBiases['hidden'])

# Split the series because the rnn cell needs time_steps features, each of shape:
hidden = tf.split(axis=0, num_or_size_splits=14, value=hidden)

lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(lstm_size, forget_bias=1.0, 
                                          state_is_tuple=True)
lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(lstm_size, forget_bias=1.0, 
                                          state_is_tuple=True)

# Stack two LSTM layers, both layers has the same shape
lstm_layers = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], 
                                          state_is_tuple=True)

lstmOutputs, _ = tf.contrib.rnn.static_rnn(lstm_layers, hidden, dtype=tf.float32)
lstmLastOutput = lstmOutputs[-1]
y_ = tf.matmul(lstmLastOutput, rnnW['output']) + rnnBiases['output']

# train and test

In [7]:
with tf.name_scope("cost"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=Y))
    # optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=
                                                  learning_rate).minimize(loss)

    # Add scalar summary for cost
    tf.summary.scalar("loss", loss)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(y_, 1)) 
    # Count correct predictions
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.summary.scalar("accuracy", accuracy)


print("net work done")

cost_history = np.empty(shape=[1],dtype=float)

with tf.Session(config=sess_config) as session:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.summary.FileWriter("data/cnn_lstm_logs", session.graph)  # for 1.0
    merged = tf.summary.merge_all()

    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)

            test_indices = np.arange(len(test_x))  # Get A Test Batch
            np.random.shuffle(test_indices)
            test_indices = test_indices[0:]
            summary, acc=session.run([merged, accuracy], 
                        feed_dict={X: test_x[test_indices], Y: test_y[test_indices]})
        print "Epoch: ",epoch," Training Loss: ",c," Training Accuracy: ", acc
        writer.add_summary(summary, epoch)  # Write summary

    print "Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y})
    writer.close()

net work done
Epoch:  0  Training Loss:  1.5546  Training Accuracy:  0.433128
Epoch:  1  Training Loss:  1.42265  Training Accuracy:  0.437477
Epoch:  2  Training Loss:  1.32914  Training Accuracy:  0.418992
Epoch:  3  Training Loss:  1.2654  Training Accuracy:  0.42153
Epoch:  4  Training Loss:  1.2338  Training Accuracy:  0.429503
Epoch:  5  Training Loss:  1.2114  Training Accuracy:  0.427691
Epoch:  6  Training Loss:  1.20749  Training Accuracy:  0.428054
Epoch:  7  Training Loss:  1.17167  Training Accuracy:  0.437115
Epoch:  8  Training Loss:  1.1529  Training Accuracy:  0.434578
Epoch:  9  Training Loss:  1.16942  Training Accuracy:  0.43784
Epoch:  10  Training Loss:  1.17405  Training Accuracy:  0.425879
Epoch:  11  Training Loss:  1.16498  Training Accuracy:  0.433853
Epoch:  12  Training Loss:  1.19389  Training Accuracy:  0.430228
Epoch:  13  Training Loss:  1.17959  Training Accuracy:  0.440015
Epoch:  14  Training Loss:  1.19844  Training Accuracy:  0.436028
Epoch:  15  T